In [ ]:
# ============================================================
# Block 1: Imports & Configuration
# ============================================================
# This block imports the required libraries for deep learning, 
# data processing, and reproducibility.
# ============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random

import yfinance as yf
from torch.utils.data import DataLoader, ConcatDataset, Subset
from sklearn.preprocessing import MinMaxScaler

# ------------------------------------------------------------
# Reproducibility setup
# ------------------------------------------------------------
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {device}")

# ------------------------------------------------------------
# Global configuration (aligned with the paper)
# ------------------------------------------------------------
config = {
    # Model dimensions
    "d_model": 64,          # shared embedding dimension
    "dropout_tech": 0.2,    # dropout for technical MLP
    "dropout_sent": 0.1,    # dropout for sentiment transformer
    "n_heads": 4,           # transformer heads
    "n_layers": 2,          # transformer layers
    
    # Training setup
    "learning_rate": 1e-3,  # Adam optimizer with cosine annealing
    "batch_size": 128,      # as specified in the paper
    "max_epochs": 200,      # max epochs with early stopping patience=20
    "early_stop_patience": 20,
    
    # Task setup
    "n_classes": 3,         # {-1, 0, +1} → mapped internally to {0,1,2}
    "delta": 0.002,         # threshold for neutral class (can be tuned)
}
print("[INFO] Global configuration loaded successfully.")

In [ ]:
# ============================================================
# Block 2: Technical Encoder
# ============================================================
# This module encodes daily technical indicators into a latent 
# representation using a two-layer MLP. 
# 
# Paper reference:
# - Input:  x_t^tech ∈ R^7 (7 domain-standard indicators)
# - Output: H_tech ∈ R^64 (shared latent dimension)
# - Architecture: 2-layer MLP with ReLU activations and dropout=0.2
# ============================================================

class TechnicalEncoder(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=64, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        """
        Encode technical indicators.
        Args:
            x: [batch_size, input_dim] tensor of technical features.
        Returns:
            [batch_size, hidden_dim] latent representation.
        """
        return self.net(x)

In [ ]:
# ============================================================
# Block 3: Sentiment Encoder
# ============================================================
# This module encodes document-level sentiment using a 
# Transformer encoder and integrates an uncertainty signal.
#
# Paper reference:
# - Input:  S_t ∈ R^{n_docs × n_models} (n_models=3: FinGPT, HAN, VADER)
# - Encoder: 2-layer Transformer, 4 heads, d_model=64, dropout=0.1
# - Pooling: mean pooling across documents (masked)
# - Uncertainty: average std across models, then averaged across documents
# - Fusion: concatenate H_sent with u_t, then project back to hidden_dim
# ============================================================

class SentimentEncoder(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, n_heads=4, n_layers=2, dropout=0.1):
        super().__init__()
        
        # Step 1: project sentiment scores into latent space
        self.embedding = nn.Linear(input_dim, hidden_dim)
        
        # Step 2: transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=n_heads,
            dim_feedforward=hidden_dim * 2,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Step 6: projection after concatenating uncertainty
        self.linear_proj = nn.Linear(hidden_dim + 1, hidden_dim)

    def compute_uncertainty(self, St, lengths):
        """
        Compute uncertainty score as described in the paper (Eq. 7).
        Args:
            St: [batch_size, n_docs, n_models]
            lengths: [batch_size] actual number of docs per sample
        Returns:
            u_t: [batch_size, 1] scalar uncertainty per sample
        """
        # std across sentiment models, shape = [B, n_docs]
        std_doc = St.std(dim=-1, unbiased=False)
        
        # mask padding documents
        mask = torch.arange(St.size(1), device=St.device)[None, :] >= lengths[:, None]
        std_doc = std_doc.masked_fill(mask, 0.0)
        
        # average across valid documents
        denom = lengths.clamp_min(1).float()
        u_t = std_doc.sum(dim=1) / denom
        return u_t.unsqueeze(1)  # [B,1]

    def forward(self, St, lengths):
        """
        Encode daily sentiment matrix.
        Args:
            St: [B, n_docs, n_models] sentiment scores
            lengths: [B] number of valid docs per sample
        Returns:
            H_sent*: [B, hidden_dim] enriched sentiment representation
                     (includes uncertainty)
        """
        # If no-news case: return None to rely on technical encoder only
        if lengths is None or (lengths == 0).all():
            return None

        # Step 1: embed sentiments
        H = self.embedding(St)  # [B, n_docs, hidden_dim]
        
        # Step 2: encode with transformer (mask padded docs)
        mask = torch.arange(St.size(1), device=St.device)[None, :] >= lengths[:, None]
        H = self.transformer(H, src_key_padding_mask=mask)
        
        # Step 3: mean pooling across valid documents
        denom = lengths.clamp_min(1).float().unsqueeze(1)
        H_sum = H.masked_fill(mask.unsqueeze(-1), 0.0).sum(dim=1)
        H_sent = H_sum / denom  # [B, hidden_dim]
        
        # Step 4: compute uncertainty
        uncertainty = self.compute_uncertainty(St, lengths)  # [B,1]
        
        # Step 5: concatenate uncertainty
        H_sent_aug = torch.cat([H_sent, uncertainty], dim=-1)  # [B, hidden_dim+1]
        
        # Step 6: project back to hidden_dim
        return self.linear_proj(H_sent_aug)

In [ ]:
# ============================================================
# Block 4: Fusion Gate
# ============================================================
# This block adaptively fuses the outputs of the technical and
# sentiment branches using a gating mechanism equivalent to 
# attention weights (Eq. 10–11 in the paper).
#
# Paper reference:
# - Input:  [H_tech, H_sent] ∈ R^d
# - Gate:   α = softmax(W_att · [H_tech || H_sent] + b), α ∈ R^2
# - Output: H_fused = α1 * H_tech + α2 * H_sent
# - No-news case: α = [1, 0] → rely entirely on technical branch
# ============================================================

class FusionGate(nn.Module):
    def __init__(self, hidden_dim=64):
        super().__init__()
        # Linear layer projects concatenated [Htech || Hsent] into 2 logits
        self.Watt = nn.Linear(hidden_dim * 2, 2)

    def forward(self, Htech, Hsent=None):
        """
        Adaptive fusion of technical and sentiment encoders.
        Args:
            Htech: [B, hidden_dim] technical representation
            Hsent: [B, hidden_dim] sentiment representation or None
        Returns:
            Hfused: [B, hidden_dim] fused representation
            alpha : [B, 2] gating weights [w_tech, w_sent]
        """
        # Case: no-news (sentiment branch missing)
        if Hsent is None:
            B, d = Htech.size()
            alpha = torch.zeros(B, 2, device=Htech.device)
            alpha[:, 0] = 1.0  # full weight to technical branch
            return Htech, alpha

        # Case: both modalities available
        combined = torch.cat([Htech, Hsent], dim=-1)  # [B, 2*hidden_dim]
        alpha = F.softmax(self.Watt(combined), dim=-1)  # [B, 2]
        
        # Weighted sum across modalities
        Hfused = alpha[:, 0].unsqueeze(1) * Htech + alpha[:, 1].unsqueeze(1) * Hsent
        return Hfused, alpha

In [ ]:
# ============================================================
# Block 5: AMFNet (Full Model)
# ============================================================
# This class implements the full Adaptive Modality Fusion 
# Network (AMFNet) as described in the paper.
#
# Paper reference:
# - Technical encoder: MLP (7 → 64)
# - Sentiment encoder: Transformer (2 layers, 4 heads, d=64)
# - Fusion gate: attention-like gating (Eq. 10–11)
# - Prediction: MLP head, softmax over 3 classes (Eq. 12)
#
# Input:
#   xtech   : [B, tech_dim]        daily technical indicators
#   St      : [B, n_docs, 3]      daily sentiment matrix (padded)
#   lengths : [B]                 number of valid docs per sample
#
# Output:
#   logits : [B, n_classes]       class scores before softmax
#   alpha  : [B, 2]               fusion weights [w_tech, w_sent]
# ============================================================

class AMFNet(nn.Module):
    def __init__(self, tech_dim=7, sent_dim=3, hidden_dim=64, n_classes=3):
        super().__init__()
        self.tech_encoder = TechnicalEncoder(input_dim=tech_dim, hidden_dim=hidden_dim)
        self.sent_encoder = SentimentEncoder(input_dim=sent_dim, hidden_dim=hidden_dim)
        self.fusion = FusionGate(hidden_dim=hidden_dim)
        self.classifier = nn.Linear(hidden_dim, n_classes)

    def forward(self, xtech, St=None, lengths=None):
        """
        Forward pass through AMFNet.
        Args:
            xtech:   [B, tech_dim]
            St:      [B, n_docs, sent_dim] sentiment matrix (optional)
            lengths: [B] number of valid documents (for masking)
        Returns:
            logits: [B, n_classes] classification scores
            alpha:  [B, 2] fusion weights
        """
        # Step 1: encode technical features
        Htech = self.tech_encoder(xtech)  # [B, hidden_dim]

        # Step 2: encode sentiment features (if available)
        Hsent = self.sent_encoder(St, lengths) if (St is not None) else None

        # Step 3: fuse modalities (handles no-news internally)
        Hfused, alpha = self.fusion(Htech, Hsent)

        # Step 4: classification head (Eq. 12)
        logits = self.classifier(Hfused)  # [B, n_classes]

        return logits, alpha

In [ ]:
# ============================================================
# Block 6: Example Usage
# ============================================================
# We test AMFNet with dummy data to ensure the forward pass 
# runs correctly. Shapes are consistent with the paper:
# - Technical features: 7 indicators
# - Sentiment matrix: n_docs × 3 models (FinGPT, HAN, VADER)
# ============================================================

# Dummy batch setup
batch_size, tech_dim, n_docs, sent_dim = 8, 7, 5, 3
xtech = torch.randn(batch_size, tech_dim).to(device)          # [B, 7]
St = torch.randn(batch_size, n_docs, sent_dim).to(device)     # [B, n_docs, 3]
lengths = torch.full((batch_size,), n_docs, dtype=torch.long).to(device)  # [B]

# Initialize model
model = AMFNet(tech_dim=tech_dim, sent_dim=sent_dim, hidden_dim=config["d_model"], n_classes=config["n_classes"]).to(device)

# Forward pass
logits, alpha = model(xtech, St, lengths)

# Logs
print("[TEST] Logits shape:", logits.shape)      # Expected: [B, 3]
print("[TEST] Fusion weights shape:", alpha.shape)  # Expected: [B, 2]
print("[TEST] Fusion weights example (first sample):", alpha[0].detach().cpu().numpy())

In [ ]:
# ============================================================
# Block X: StockDataset
# ============================================================
# This dataset prepares daily samples for AMFNet:
#   - Technical indicators (xtech): 7 features
#   - Sentiment matrix (St): [n_docs_max × n_models] (padded)
#   - lengths: number of valid documents per sample
#   - Label y: {0=Neutral, 1=Buy, 2=Sell}
#   - Realized next-day returns (r_t1): float
#
# Paper reference:
# - Labels defined by Eq. (2) with threshold delta
# - Sentiment scores normalized into [-1, +1] (Eq. 3–4)
# - Zero-doc case handled explicitly (fallback to technical branch)
# ============================================================

from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch

class StockDataset(Dataset):
    def __init__(self, df, n_docs_max=10, n_models=3, delta=0.002):
        self.df = df.copy()
        self.n_docs_max = n_docs_max
        self.n_models = n_models
        self.delta = delta

        # --- Technical indicators (7 features) ---
        self.xtech = self.df[["RSI", "MACD", "VWAP", "BB_Width", "ATR", "OBV", "ROC"]].values.astype(np.float32)

        # --- Next-day returns and labels (Eq. 2) ---
        close = self.df["Close"].values
        r_t1 = (np.roll(close, -1) - close) / close

        # Labeling with neutral threshold delta
        y = np.zeros_like(r_t1, dtype=np.int64)
        y[r_t1 > delta] = 1  # Buy
        y[r_t1 < -delta] = 2 # Sell

        # Drop last row (no next-day return available)
        self.y = y[:-1]
        self.r_t1 = r_t1[:-1].astype(np.float32)
        self.xtech = self.xtech[:-1]

        # --- Sentiment matrices (dummy data here) ---
        n_days = len(self.y)
        self.St = []
        self.lengths = []

        # MinMax scaler for normalization ([-1,1]), one per model
        scalers = [MinMaxScaler(feature_range=(-1, 1)) for _ in range(n_models)]

        for _ in range(n_days):
            # Variable number of documents [0..n_docs_max]
            n_docs = np.random.randint(0, n_docs_max + 1)
            self.lengths.append(n_docs)

            # Initialize matrix with padding
            mat = np.zeros((n_docs_max, n_models), dtype=np.float32)

            if n_docs > 0:
                # Simulate raw sentiment scores
                raw = np.random.uniform(-1, 1, size=(n_docs, n_models))

                # Normalize each model column with its own scaler (fit_transform here just for dummy data)
                normed = np.zeros_like(raw)
                for j in range(n_models):
                    normed[:, j] = scalers[j].fit_transform(raw[:, j].reshape(-1, 1)).squeeze()

                mat[:n_docs] = normed

            self.St.append(mat)

        self.St = np.stack(self.St, axis=0)
        self.lengths = np.array(self.lengths, dtype=np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        xtech = torch.tensor(self.xtech[idx], dtype=torch.float32)     # [7]
        St = torch.tensor(self.St[idx], dtype=torch.float32)           # [n_docs_max, n_models]
        length = torch.tensor(self.lengths[idx], dtype=torch.long)     # scalar
        y = torch.tensor(self.y[idx], dtype=torch.long).squeeze()      # scalar
        return xtech, St, length, y

In [ ]:
# ============================================================
# Block 8: Training Setup
# ============================================================
# Parameters:
#   - 12 US large-cap tickers (2015–2024)
#   - Technical indicators: 7 features
#   - Sentiment: 3 models (dummy matrices here, normalized)
#   - Label threshold delta=0.002
#   - Rolling window: 24m train / 3m val / 3m test (12 folds)
#   - Hidden dim=64, batch=128, max_epochs=200, early stop=20
#   - Optimizer: Adam, lr=1e-3, CosineAnnealingLR scheduler
# ============================================================

from torch.utils.data import DataLoader, ConcatDataset, Subset
from sklearn.preprocessing import StandardScaler
import yfinance as yf
import datetime

# ----------------------------
# Step 1: Parameters
# ----------------------------
tickers = ["AAPL","NVDA","JPM","BAC","JNJ","PFE","XOM","CVX","BA","CAT","KO","PG"]

start_date = "2015-01-01"
end_date   = "2024-01-01"
n_docs_max, n_models = 20, 3
delta = config["delta"]

batch_size = config["batch_size"]
max_epochs = config["max_epochs"]
patience   = config["early_stop_patience"]

# ----------------------------
# Step 2: Technical indicators
# ----------------------------
def compute_indicators(df):
    df = df.copy()
    # RSI
    delta_p = df["Close"].diff()
    gain = (delta_p.where(delta_p > 0, 0)).rolling(14).mean()
    loss = (-delta_p.where(delta_p < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26

    # VWAP
    df["VWAP"] = (df["Close"] * df["Volume"]).cumsum() / df["Volume"].cumsum()

    # Bollinger Band Width
    mean20 = df["Close"].rolling(20).mean()
    std20 = df["Close"].rolling(20).std()
    df["BB_Width"] = 4 * std20

    # ATR
    high_low = df["High"] - df["Low"]
    high_close = (df["High"] - df["Close"].shift()).abs()
    low_close = (df["Low"] - df["Close"].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df["ATR"] = tr.rolling(14).mean()

    # OBV
    obv = (np.sign(df["Close"].diff()) * df["Volume"]).fillna(0).cumsum()
    df["OBV"] = obv

    # ROC
    df["ROC"] = df["Close"].pct_change(periods=12)

    return df.dropna()

# ----------------------------
# Step 3: Rolling Window Splits
# ----------------------------
def rolling_windows(df, train_months=24, val_months=3, test_months=3):
    """Generate rolling windows: 24m train / 3m val / 3m test."""
    folds = []
    start = df.index.min()
    while True:
        train_start = start
        train_end   = train_start + pd.DateOffset(months=train_months)
        val_end     = train_end   + pd.DateOffset(months=val_months)
        test_end    = val_end     + pd.DateOffset(months=test_months)

        if test_end > df.index.max():
            break

        train_idx = (df.index >= train_start) & (df.index < train_end)
        val_idx   = (df.index >= train_end)   & (df.index < val_end)
        test_idx  = (df.index >= val_end)     & (df.index < test_end)

        folds.append((df[train_idx], df[val_idx], df[test_idx]))
        start = val_end  # slide window

    return folds

# ----------------------------
# Step 4: Build datasets
# ----------------------------
all_folds = []
for t in tickers:
    df = yf.download(t, start=start_date, end=end_date, auto_adjust=False)
    df = compute_indicators(df).dropna()

    folds = rolling_windows(df)
    for train_df, val_df, test_df in folds:
        # scale on train only
        scaler = StandardScaler()
        scaler.fit(train_df[["RSI","MACD","VWAP","BB_Width","ATR","OBV","ROC"]])
        for part_df in [train_df, val_df, test_df]:
            part_df.loc[:, ["RSI","MACD","VWAP","BB_Width","ATR","OBV","ROC"]] = scaler.transform(
                part_df[["RSI","MACD","VWAP","BB_Width","ATR","OBV","ROC"]]
            )

        train_set = StockDataset(train_df, n_docs_max=n_docs_max, n_models=n_models, delta=delta)
        val_set   = StockDataset(val_df,   n_docs_max=n_docs_max, n_models=n_models, delta=delta)
        test_set  = StockDataset(test_df,  n_docs_max=n_docs_max, n_models=n_models, delta=delta)

        all_folds.append((train_set, val_set, test_set))

print(f"[INFO] Generated {len(all_folds)} rolling folds across {len(tickers)} tickers.")

# ----------------------------
# Step 5: DataLoader builder
# ----------------------------
def my_collate_fn(batch):
    xtechs, Sts, lengths, ys = zip(*batch)
    return (torch.stack(xtechs),
            torch.stack(Sts),
            torch.stack(lengths),
            torch.stack(ys))

def make_loaders(train_set, val_set, test_set):
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)
    val_loader   = DataLoader(val_set,   batch_size=batch_size, shuffle=False, collate_fn=my_collate_fn)
    test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False, collate_fn=my_collate_fn)
    return train_loader, val_loader, test_loader

In [ ]:
# ============================================================
# Block 9: Training & Evaluation Loop
# ============================================================
# This block implements the training pipeline for AMFNet:
#   - Optimizer: Adam (lr=1e-3)
#   - Scheduler: CosineAnnealingLR
#   - Loss: CrossEntropyLoss
#   - Early stopping: patience=20 on validation F1
#   - Metrics: Accuracy, Macro-F1
#   - Protocol: rolling windows (24m train / 3m val / 3m test)
# ============================================================

from sklearn.metrics import accuracy_score, f1_score
import copy
import torch

def evaluate(model, loader, criterion):
    """Evaluate model on given loader."""
    model.eval()
    total_loss, all_preds, all_labels = 0, [], []
    with torch.no_grad():
        for xtech, St, lengths, y in loader:
            xtech, St, lengths, y = xtech.to(device), St.to(device), lengths.to(device), y.to(device)
            logits, _ = model(xtech, St, lengths)
            loss = criterion(logits, y)

            total_loss += loss.item()
            all_preds.extend(logits.argmax(dim=1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    return total_loss/len(loader), accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average="macro")

def train_one_fold(train_set, val_set, test_set, fold_id=0):
    """Train AMFNet on a single rolling window fold."""
    print(f"\n[INFO] Starting Fold {fold_id+1}/{len(all_folds)}")

    # DataLoaders
    train_loader, val_loader, test_loader = make_loaders(train_set, val_set, test_set)

    # Model, optimizer, scheduler
    model = AMFNet(tech_dim=7, sent_dim=3, hidden_dim=config["d_model"], n_classes=config["n_classes"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config["max_epochs"])

    # Early stopping
    best_val_f1, best_state, patience_counter = -1, None, 0

    # Training loop
    for epoch in range(config["max_epochs"]):
        model.train()
        total_loss, all_preds, all_labels = 0, [], []

        for xtech, St, lengths, y in train_loader:
            xtech, St, lengths, y = xtech.to(device), St.to(device), lengths.to(device), y.to(device)
            logits, _ = model(xtech, St, lengths)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            all_preds.extend(logits.argmax(dim=1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

        train_loss = total_loss / len(train_loader)
        train_acc = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average="macro")

        # Validation
        val_loss, val_acc, val_f1 = evaluate(model, val_loader, criterion)
        scheduler.step()

        print(f"Epoch {epoch+1}/{config['max_epochs']} | "
              f"Train Loss={train_loss:.4f} Acc={train_acc:.4f} F1={train_f1:.4f} || "
              f"Val Loss={val_loss:.4f} Acc={val_acc:.4f} F1={val_f1:.4f}")

        # Early stopping check
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= config["early_stop_patience"]:
            print(f"[INFO] Early stopping triggered at epoch {epoch+1}")
            break

    # Load best state
    model.load_state_dict(best_state)

    # Final Test
    test_loss, test_acc, test_f1 = evaluate(model, test_loader, criterion)
    print(f"[RESULT] Fold {fold_id+1}: Test Loss={test_loss:.4f} Acc={test_acc:.4f} F1={test_f1:.4f}")

    return test_loss, test_acc, test_f1

In [ ]:
# ============================================================
# Block 10: Run All Rolling Window Folds
# ============================================================
# Train and evaluate AMFNet across all rolling windows.
# Report mean ± std across folds (as in the paper).
# ============================================================

all_results = []
for i, (train_set, val_set, test_set) in enumerate(all_folds):
    test_loss, test_acc, test_f1 = train_one_fold(train_set, val_set, test_set, fold_id=i)
    all_results.append((test_loss, test_acc, test_f1))

all_results = np.array(all_results)
mean_results = all_results.mean(axis=0)
std_results  = all_results.std(axis=0)

print("\n[SUMMARY] Final Results across folds:")
print(f"  Test Loss = {mean_results[0]:.4f} ± {std_results[0]:.4f}")
print(f"  Test Acc  = {mean_results[1]:.4f} ± {std_results[1]:.4f}")
print(f"  Test F1   = {mean_results[2]:.4f} ± {std_results[2]:.4f}")

In [ ]:
# ============================================================
# Block A: Classification Metrics (Accuracy, Precision, Recall, F1)
# ============================================================
# This block evaluates AMFNet on classification performance 
# exactly as reported in the article (Section IV.C).
#
# Metrics:
#   - Global Accuracy
#   - Per-class Precision, Recall, F1
# Classes:
#   {0 = Neutral, 1 = Buy, 2 = Sell}
# ============================================================

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

@torch.no_grad()
def evaluate_classification_metrics(model, loader, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Evaluate classification metrics (Accuracy, Precision, Recall, F1) 
    on a given dataloader.
    
    Args:
        model  : trained AMFNet model
        loader : DataLoader (val/test split)
        device : "cuda" or "cpu"
    Returns:
        dict with accuracy, per-class precision/recall/F1, 
        and raw predictions/labels.
    """
    model.eval()
    all_preds, all_labels = [], []

    for xtech, St, lengths, y in loader:
        xtech, St, lengths, y = xtech.to(device), St.to(device), lengths.to(device), y.to(device)
        logits, _ = model(xtech, St, lengths)
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = y.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

    # Global Accuracy
    acc = accuracy_score(all_labels, all_preds)

    # Per-class metrics (0=Neutral, 1=Buy, 2=Sell)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average=None, labels=[0,1,2], zero_division=0
    )

    # Logging
    print("== Classification Results ==")
    print(f"Accuracy: {acc:.4f}\n")
    print("Per-class (0=Neutral, 1=Buy, 2=Sell):")
    print("Class | Precision | Recall | F1")
    for cls, p, r, f in zip([0,1,2], precision, recall, f1):
        print(f"  {cls:5d} |   {p:.4f}   |  {r:.4f}  | {f:.4f}")

    return {
        "accuracy": acc,
        "precision_per_class": precision.tolist(),
        "recall_per_class": recall.tolist(),
        "f1_per_class": f1.tolist(),
        "y_true": all_labels,
        "y_pred": all_preds,
    }

# ============================================================
# Example usage (after training one fold)
# ============================================================

train_loader, val_loader, test_loader = make_loaders(train_set, val_set, test_set)

metrics_test = evaluate_classification_metrics(model, test_loader, device)

In [ ]:
# ============================================================
# Block B: Statistical Validation (multi-seed + 95% CIs)
# ============================================================
# - Train AMFNet multiple times with different seeds
# - Select best checkpoint per seed by validation F1
# - Report mean ± std and 95% CI for Accuracy and F1
# ============================================================

import numpy as np
import torch
import torch.nn as nn

def set_global_seed(seed: int):
    """Ensure reproducibility across runs."""
    import random, numpy as _np, torch as _torch
    random.seed(seed); _np.random.seed(seed); _torch.manual_seed(seed)
    if _torch.cuda.is_available():
        _torch.cuda.manual_seed_all(seed)

def ci95(x):
    """95% confidence interval under normal approximation."""
    x = np.asarray(x, dtype=float)
    mean = x.mean()
    std = x.std(ddof=1) if len(x) > 1 else 0.0
    half = 1.96 * (std / np.sqrt(len(x))) if len(x) > 1 else 0.0
    return mean, std, (mean - half, mean + half)

def train_one_run(train_loader, val_loader, test_loader, device, epochs=200, lr=1e-3):
    """
    Train AMFNet once with a given seed.
    Save best checkpoint based on validation F1.
    """
    # Fresh model
    model = AMFNet(tech_dim=7, sent_dim=3, hidden_dim=64).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_f1 = -1.0
    best_state = None

    for _ in range(epochs):
        # --- training ---
        model.train()
        for xtech, St, lengths, y in train_loader:
            xtech, St, lengths, y = xtech.to(device), St.to(device), lengths.to(device), y.to(device)
            logits, _ = model(xtech, St, lengths)
            loss = criterion(logits, y)
            optimizer.zero_grad(); loss.backward(); optimizer.step()

        # --- validation (F1) ---
        model.eval()
        val = evaluate_classification_metrics(model, val_loader, device)
        f1_val = np.mean(val["f1_per_class"])  # macro-F1 across 3 classes
        if f1_val > best_val_f1:
            best_val_f1 = f1_val
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}

    # Reload best checkpoint
    if best_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_state.items()})

    # --- final test evaluation ---
    test = evaluate_classification_metrics(model, test_loader, device)
    f1_test = np.mean(test["f1_per_class"])
    return test["accuracy"], f1_test

def statistical_validation(train_loader, val_loader, test_loader, device,
                           seeds=(42, 43, 44, 45, 46), epochs=200):
    """
    Run multiple seeds and report mean ± std and 95% CI.
    """
    accs, f1s = [], []
    for s in seeds:
        set_global_seed(s)
        acc, f1 = train_one_run(train_loader, val_loader, test_loader, device, epochs=epochs)
        accs.append(acc); f1s.append(f1)

    acc_mean, acc_std, acc_ci = ci95(accs)
    f1_mean, f1_std, f1_ci = ci95(f1s)

    print("== Statistical Validation (multi-seed) ==")
    print(f"Accuracy : {acc_mean:.4f} ± {acc_std:.4f} (95% CI: {acc_ci[0]:.4f} – {acc_ci[1]:.4f})")
    print(f"F1       : {f1_mean:.4f} ± {f1_std:.4f} (95% CI: {f1_ci[0]:.4f} – {f1_ci[1]:.4f})")

    return {"accs": accs, "f1s": f1s, "acc_ci": acc_ci, "f1_ci": f1_ci}

# ----------------------------
# Example usage (after defining loaders for a fold):
# ----------------------------
stats = statistical_validation(train_loader, val_loader, test_loader,
                               device, seeds=[11,12,13,14,15], epochs=50)  # shorter for demo

In [ ]:
# ============================================================
# Block C: Portfolio-based Evaluation
# ============================================================
# Converts class predictions into trading signals {-1, 0, +1},
# computes portfolio metrics (CR, Sharpe, MDD, HR),
# both with and without transaction costs (10 bps).
# ============================================================

import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
import yfinance as yf

def signals_from_preds(y_pred):
    """Map class predictions {0=Neutral, 1=Buy, 2=Sell} -> trading signals {0,+1,-1}"""
    y_pred = np.asarray(y_pred)
    sig = np.zeros_like(y_pred, dtype=np.int8)
    sig[y_pred == 1] = +1   # Buy
    sig[y_pred == 2] = -1   # Sell
    return sig

def portfolio_metrics(strategy_rets):
    """Compute CR, Sharpe, MDD, HR from daily strategy returns."""
    strategy_rets = np.asarray(strategy_rets, dtype=float)
    strategy_rets = np.nan_to_num(strategy_rets, nan=0.0, posinf=0.0, neginf=0.0)
    strategy_rets = np.clip(strategy_rets, -0.5, 0.5)  # safeguard

    # Cumulative return
    cum_curve = np.cumprod(1.0 + strategy_rets)
    cr = cum_curve[-1] - 1.0

    # Sharpe ratio
    sr = strategy_rets.mean() / (strategy_rets.std(ddof=1) + 1e-12)

    # Max Drawdown
    peak = np.maximum.accumulate(cum_curve)
    mdd = (cum_curve / peak - 1.0).min()

    # Hit Ratio
    hr = (strategy_rets > 0).mean()

    return {"CR": cr, "Sharpe": sr, "MDD": mdd, "HitRatio": hr}

@torch.no_grad()
def evaluate_portfolio(model, loader, base_dataset, costs_bps=10,
                       device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Evaluate portfolio performance using AMFNet predictions.
    Returns metrics with and without transaction costs.
    """
    model.eval()
    preds, rets = [], []
    idx_offset = getattr(loader.dataset, "indices", None)
    running_index = 0

    for xtech, St, length, y in loader:
        xtech, St, length = xtech.to(device), St.to(device), length.to(device)
        logits, _ = model(xtech, St, length)
        batch_pred = logits.argmax(dim=1).cpu().numpy()
        preds.extend(batch_pred)

        bsz = len(batch_pred)
        if idx_offset is not None:
            batch_idx = loader.dataset.indices[running_index:running_index+bsz]
            batch_rets = base_dataset.r_t1[batch_idx]
        else:
            batch_rets = base_dataset.r_t1[running_index:running_index+bsz]
        rets.extend(batch_rets)
        running_index += bsz

    preds = np.asarray(preds)
    rets = np.asarray(rets, dtype=float)

    # Strategy returns without costs
    sig = signals_from_preds(preds)
    strat_rets_raw = sig * rets

    # Strategy returns with transaction costs
    costs = costs_bps / 10000.0
    sig_shift = np.roll(sig, 1); sig_shift[0] = 0
    trades = (sig != sig_shift).astype(int)
    strat_rets_cost = strat_rets_raw - trades * costs

    # Portfolio metrics
    no_cost = portfolio_metrics(strat_rets_raw)
    with_cost = portfolio_metrics(strat_rets_cost)

    print("== Portfolio metrics (no costs) ==")
    print(f"CR: {no_cost['CR']*100:6.2f}% | Sharpe: {no_cost['Sharpe']:.2f} | "
          f"MDD: {no_cost['MDD']*100:6.2f}% | HR: {no_cost['HitRatio']*100:5.1f}%")
    print("== Portfolio metrics (10 bps costs) ==")
    print(f"CR: {with_cost['CR']*100:6.2f}% | Sharpe: {with_cost['Sharpe']:.2f} | "
          f"MDD: {with_cost['MDD']*100:6.2f}% | HR: {with_cost['HitRatio']*100:5.1f}%")

    return {
        "no_cost": {**no_cost, "strategy_rets": strat_rets_raw},
        "with_cost": {**with_cost, "strategy_rets": strat_rets_cost},
        "signals": sig,
        "rets": rets
    }

# ----------------------------
# Example usage (AAPL demo)
# ----------------------------
df_aapl = yf.download("AAPL", start="2015-01-01", end="2024-01-01", auto_adjust=False)
df_aapl = compute_indicators(df_aapl)  # function from Block 8
ds_aapl = StockDataset(df_aapl, n_docs_max=20, n_models=3, delta=0.002)

# use last 15% as test set
test_set_aapl = Subset(ds_aapl, range(int(0.85*len(ds_aapl)), len(ds_aapl)))
test_loader_aapl = DataLoader(test_set_aapl, batch_size=64, shuffle=False)

portfolio_results = evaluate_portfolio(model, test_loader_aapl, ds_aapl, costs_bps=10, device=device)

In [ ]:
# ============================================================
# Block Cbis: Multi-ticker Portfolio Evaluation
# ============================================================
# Runs evaluate_portfolio for each ticker and aggregates results
# across all tickers: mean ± std of CR, Sharpe, MDD, HR.
# ============================================================

def evaluate_portfolio_multi(model, tickers, start_date="2015-01-01", end_date="2024-01-01",
                             costs_bps=10, device="cuda" if torch.cuda.is_available() else "cpu"):
    results_no_cost, results_with_cost = [], []

    for t in tickers:
        print(f"\n=== Evaluating {t} ===")
        df = yf.download(t, start=start_date, end=end_date, auto_adjust=False)
        df = compute_indicators(df)  # from Block 8

        ds_t = StockDataset(df, n_docs_max=20, n_models=3, delta=0.002)
        test_set_t = Subset(ds_t, range(int(0.85*len(ds_t)), len(ds_t)))  # last 15% = test
        test_loader_t = DataLoader(test_set_t, batch_size=64, shuffle=False)

        res = evaluate_portfolio(model, test_loader_t, ds_t, costs_bps=costs_bps, device=device)
        results_no_cost.append(res["no_cost"])
        results_with_cost.append(res["with_cost"])

    def agg_metrics(res_list):
        metrics = {}
        for key in ["CR", "Sharpe", "MDD", "HitRatio"]:
            values = [r[key] for r in res_list]
            metrics[key] = {
                "mean": np.mean(values),
                "std": np.std(values, ddof=1)
            }
        return metrics

    agg_no_cost = agg_metrics(results_no_cost)
    agg_with_cost = agg_metrics(results_with_cost)

    print("\n=== Aggregated Portfolio Results (across tickers) ===")
    print("No Costs:")
    for k, v in agg_no_cost.items():
        print(f"{k:8s}: {v['mean']:.4f} ± {v['std']:.4f}")
    print("\nWith Costs (10 bps):")
    for k, v in agg_with_cost.items():
        print(f"{k:8s}: {v['mean']:.4f} ± {v['std']:.4f}")

    return {"no_cost": agg_no_cost, "with_cost": agg_with_cost}

# ----------------------------
# Example usage on multiple tickers
# ----------------------------
tickers = ["AAPL","MSFT","GOOG","AMZN","META","NVDA","TSLA","JPM","BAC","XOM","PFE","INTC"]
portfolio_summary = evaluate_portfolio_multi(model, tickers, costs_bps=10, device=device)

In [ ]:
# ============================================================
# Block D: Average Modality Attention per Sector
# ============================================================
# Computes average attention weights (α_tech, α_sent) per sector
# for the given tickers and plots a grouped bar chart.
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

@torch.no_grad()
def evaluate_and_plot_attention_by_sector(model, tickers, sector_map,
                                          start_date="2015-01-01", end_date="2024-01-01",
                                          device="cuda" if torch.cuda.is_available() else "cpu",
                                          title="Average Modality Attention by Sector (2023–2024)"):
    results = []

    for t in tickers:
        df = yf.download(t, start=start_date, end=end_date, auto_adjust=False)
        df = compute_indicators(df).dropna()

        ds_t = StockDataset(df, n_docs_max=20, n_models=3, delta=0.002)
        test_set_t = Subset(ds_t, range(int(0.85*len(ds_t)), len(ds_t)))  # last 15% as test
        test_loader_t = DataLoader(test_set_t, batch_size=64, shuffle=False)

        model.eval()
        alphas = []

        for xtech, St, lengths, y in test_loader_t:
            xtech, St, lengths = xtech.to(device), St.to(device), lengths.to(device)
            _, alpha = model(xtech, St, lengths)  # alpha: [B, 2]
            alphas.append(alpha.cpu().numpy())

        if alphas:
            alphas = np.concatenate(alphas, axis=0)
            mean_alpha = alphas.mean(axis=0)  # [2] -> [α_tech, α_sent]
            results.append({
                "Ticker": t,
                "Sector": sector_map[t],  # strict mapping
                "Alpha_tech": mean_alpha[0],
                "Alpha_sent": mean_alpha[1]
            })

    # Aggregate per sector
    df_alpha = pd.DataFrame(results)
    df_sector = df_alpha.groupby("Sector")[["Alpha_tech", "Alpha_sent"]].mean().reset_index()

    print("\n== Average Modality Attention per Sector ==")
    print(df_sector)

    # -------- Plot grouped bar chart --------
    sectors = df_sector["Sector"].tolist()
    alpha_sent = df_sector["Alpha_sent"].values
    alpha_tech = df_sector["Alpha_tech"].values

    x = np.arange(len(sectors))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10,6))
    ax.bar(x - width/2, alpha_sent, width, label="Sentiment Attention")
    ax.bar(x + width/2, alpha_tech, width, label="Technical Attention")

    ax.set_ylabel("Average Attention Weight")
    ax.set_xlabel("Sector")
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(sectors, rotation=20, ha="right")
    ax.set_ylim(0, 1)
    ax.legend()

    plt.tight_layout()
    plt.show()

    return df_alpha, df_sector

# ----------------------------
# Example usage
# ----------------------------
tickers = ["AAPL","NVDA","JPM","BAC","JNJ","PFE","XOM","CVX","BA","CAT","KO","PG"]
sector_map = {
    # Technology
    "AAPL": "Technology",
    "NVDA": "Technology",

    # Finance
    "JPM": "Finance",
    "BAC": "Finance",

    # Healthcare
    "JNJ": "Healthcare",
    "PFE": "Healthcare",

    # Energy
    "XOM": "Energy",
    "CVX": "Energy",

    # Industrials
    "BA": "Industrials",
    "CAT": "Industrials",

    # Consumer Staples
    "KO": "Consumer Staples",
    "PG": "Consumer Staples"
}
df_alpha, df_sector = evaluate_and_plot_attention_by_sector(model, tickers, sector_map, device=device)

In [ ]:
# ============================================================
# Block E: Quarterly F1-score + Market Regime Analysis
# ============================================================
# 1. Computes quarterly F1-score of AMFNet over the 2023–2024 test period.
# 2. Computes average modality attention weights (α_sent, α_tech)
#    under different market regimes (Stable vs Volatile),
#    and reports the Modality Ratio (α_sent / α_tech).
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score

@torch.no_grad()
def quarterly_f1_scores(model, base_dataset, test_loader, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Compute quarterly macro-F1 scores for the test set.
    Args:
        model        : trained AMFNet model
        base_dataset : StockDataset (provides aligned dates)
        test_loader  : DataLoader for the test split
        device       : "cuda" or "cpu"
    Returns:
        pandas.Series with quarterly macro-F1
    """
    model.eval()
    all_preds, all_labels, all_dates = [], [], []

    # Dates aligned with returns (shifted because of r_t1)
    dates = base_dataset.df.index[1:]
    test_indices = test_loader.dataset.indices if hasattr(test_loader.dataset, "indices") else range(len(test_loader.dataset))

    for batch_i, (xtech, St, lengths, y) in enumerate(test_loader):
        xtech, St, lengths, y = xtech.to(device), St.to(device), lengths.to(device), y.to(device)
        logits, _ = model(xtech, St, lengths)
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = y.cpu().numpy()

        start = batch_i * test_loader.batch_size
        end = start + len(preds)
        batch_dates = dates[test_indices[start:end]]

        all_preds.extend(preds)
        all_labels.extend(labels)
        all_dates.extend(batch_dates)

    df_eval = pd.DataFrame({
        "Date": pd.to_datetime(all_dates),
        "y_true": all_labels,
        "y_pred": all_preds
    })
    df_eval["Quarter"] = df_eval["Date"].dt.to_period("Q")

    # Compute macro-F1 per quarter
    f1_quarter = df_eval.groupby("Quarter").apply(
        lambda g: f1_score(g["y_true"], g["y_pred"], average="macro")
    )
    return f1_quarter

def plot_quarterly_f1(f1_quarter, title="Quarterly F1-score of AMFNet (2023–2024)"):
    """
    Plot a line chart of quarterly macro-F1 scores.
    """
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(f1_quarter.index.astype(str), f1_quarter.values, marker="o")
    ax.set_title(title)
    ax.set_xlabel("Quarter")
    ax.set_ylabel("F1-score")
    ax.set_ylim(0.84, 0.89)
    ax.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

@torch.no_grad()
def attention_by_market_regime(model, base_dataset, test_loader, 
                               stable_period=("2023Q1", "2023Q1"), 
                               volatile_period=("2023Q2", "2024Q1"),
                               device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Compute average α_sent and modality ratio (α_sent/α_tech)
    for stable vs volatile market regimes.
    Args:
        model        : trained AMFNet
        base_dataset : StockDataset with aligned dates
        test_loader  : DataLoader for test split
        stable_period: tuple ("start_quarter", "end_quarter")
        volatile_period: tuple ("start_quarter", "end_quarter")
    Returns:
        pandas.DataFrame with regime statistics
    """
    model.eval()
    alphas, all_dates = [], []

    dates = base_dataset.df.index[1:]
    test_indices = test_loader.dataset.indices if hasattr(test_loader.dataset, "indices") else range(len(test_loader.dataset))

    running_index = 0
    for xtech, St, lengths, y in test_loader:
        xtech, St, lengths = xtech.to(device), St.to(device), lengths.to(device)
        _, alpha = model(xtech, St, lengths)  # [B,2]
        bsz = len(alpha)
        batch_dates = dates[test_indices[running_index:running_index+bsz]]
        running_index += bsz

        alphas.append(alpha.cpu().numpy())
        all_dates.extend(batch_dates)

    alphas = np.concatenate(alphas, axis=0)
    df_alpha = pd.DataFrame(alphas, columns=["Alpha_tech","Alpha_sent"])
    df_alpha["Date"] = pd.to_datetime(all_dates)
    df_alpha["Quarter"] = df_alpha["Date"].dt.to_period("Q")

    # Stable period
    df_stable = df_alpha[df_alpha["Quarter"].astype(str).between(stable_period[0], stable_period[1])]
    alpha_sent_stable = df_stable["Alpha_sent"].mean()
    alpha_tech_stable = df_stable["Alpha_tech"].mean()
    ratio_stable = alpha_sent_stable / (alpha_tech_stable + 1e-12)

    # Volatile period
    df_vol = df_alpha[df_alpha["Quarter"].astype(str).between(volatile_period[0], volatile_period[1])]
    alpha_sent_vol = df_vol["Alpha_sent"].mean()
    alpha_tech_vol = df_vol["Alpha_tech"].mean()
    ratio_vol = alpha_sent_vol / (alpha_tech_vol + 1e-12)

    # Create results table
    table = pd.DataFrame([
        {"Market Period": "Stable (Q1 2023)", "α_sent": round(alpha_sent_stable,2), "ModalityRatio": round(ratio_stable,2)},
        {"Market Period": "Volatile (Q2 2023–Q1 2024)", "α_sent": round(alpha_sent_vol,2), "ModalityRatio": round(ratio_vol,2)}
    ])
    print("\n== Average attention weights under market regimes ==")
    print(table.to_string(index=False))
    return table

# Quarterly F1-score
f1_quarter = quarterly_f1_scores(model, ds_aapl, test_loader_aapl, device=device)
plot_quarterly_f1(f1_quarter)

# Market regime attention
table_regimes = attention_by_market_regime(model, ds_aapl, test_loader_aapl, device=device)